In [10]:
import os

In [11]:
%pwd

'c:\\Users\\Venks\\Desktop\\Project\\MLOPS_Bootcamp\\Loan_Risk_Assessment_Platform\\research'

In [12]:
os.chdir("../")

In [13]:
%pwd

'c:\\Users\\Venks\\Desktop\\Project\\MLOPS_Bootcamp\\Loan_Risk_Assessment_Platform'

Updating entity which will contain the return type of the get_data_ingestion function under configuration.

In [14]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

Imported necessary files from constants which contains the path details and imported common files from utils

In [15]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

Configuration Manager: It will read the path of yaml file config, param and schema and create directory under __init__ constructor and has one more function get data ingestion fuction whose returrn type was earlier defined as an entity using dataclass library

In [16]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

data ingestion component contains the class of Data ingestion under which __init__ contains the DATAingestion configration (paths), download file, extract zip file, request here is used for downloading the data

In [17]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [18]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

Data Ingestion pipeline initializes the config object with configuration manager class, post that data ingestion component class is called within which methods of download_file and extract_zip_file is called. 

In [20]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-11-11 20:32:21,045: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-11-11 20:32:21,052: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-11 20:32:21,054: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-11-11 20:32:21,054: INFO: common: created directory at: artifacts]
[2025-11-11 20:32:21,054: INFO: common: created directory at: artifacts/data_ingestion]
[2025-11-11 20:32:24,095: INFO: 2366278714: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 15357919
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "67b0cce4ad06c589b5f0c1a55700938614c590ffc5ab3a5553e2d9572ea21f1e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 5FA1:3D8B8D:687107:E85005:69134FFE
Accept-Ranges: bytes
Date: Tue,